# 01. LLM単体でのテキスト生成 (ベースライン)

このノートブックでは、ファインチューニングやRAGを行っていない、素のLLM（大規模言語モデル）がどのようにテキストを生成するかを体験します。

## 事前準備

Google Colabで実行する場合、メニューの「ランタイム」→「ランタイムのタイプを変更」で、ハードウェアアクセラレータが「T4 GPU」になっていることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
from google.colab import drive

# Google Driveのマウント
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

# リポジトリパスの設定と移動
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone https://github.com/akio-kobayashi/llm_lab.git

os.chdir(repo_path)

# 必要なライブラリのインストール（Colab環境用）
print('ライブラリをインストールしています...（数分かかります）')
!pip install -q -U transformers==4.41.2 accelerate==0.30.1 bitsandbytes==0.46.1 sentence-transformers==2.7.0 faiss-gpu peft==0.10.0 trl==0.8.6 datasets==2.19.0 gradio==4.31.5


# srcディレクトリにパスを通す
if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

# 共通関数のインポート
try:
    from src.common import load_llm, generate_text
    print('共通モジュールのインポートが完了しました。')
except ImportError:
    print('共通モジュールのインポートに失敗しました。00_setup_common.ipynb を実行したか確認してください。')

## モデルのロード

演習で共通して使用するLLMをロードします。`common.py` で定義された `load_llm` 関数を呼び出します。
モデルのロードには数分かかります。

In [ ]:
# 編集禁止セル
# グローバル変数としてモデルとトークナイザを保持
model = None
tokenizer = None

def load_model_globally():
    global model, tokenizer
    if model is None or tokenizer is None:
        try:
            model, tokenizer = load_llm(use_4bit=True)
        except Exception as e:
            print(f'モデルのロード中にエラーが発生しました: {e}')
            print('GPUメモリが不足している可能性があります。ランタイムを再起動して、このセルから再実行してください。')
    else:
        print("モデルは既にロードされています。")

load_model_globally()

## テキスト生成の体験

ロードしたLLMに様々な質問を投げかけて、どのような回答が返ってくるかを見てみましょう。

### 演習1: 同じ質問を繰り返す

以下のセルの `question` に好きな質問を入れて、**複数回実行**してみてください。
毎回、少しずつ回答内容が変わる（＝生成が揺れる）ことを確認しましょう。これは、LLMが確率に基づいて次の単語を予測しているために起こる現象です。

In [ ]:
# --- ここを編集 --- #
question = "日本のおすすめの観光地を3つ教えてください。"
# --- 編集ここまで --- #

# 編集禁止セル
if model and tokenizer:
    # プロンプトテンプレートを適用
    prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n\n### 応答:\n"
    
    generated_text = generate_text(model, tokenizer, prompt)
    
    print("--- 質問 ---")
    print(question)
    print("\n--- LLMの回答 ---")
    # 応答部分のみを切り出して表示
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

### 演習2: LLMが知らないはずの質問をする

LLMは、学習データに含まれていない、非常に新しい情報や、架空の情報については答えることができません。
しかし、それらしい嘘の回答（**ハルシネーション**）を生成してしまうことがあります。

以下の `question` に、LLMが知らないであろう質問（例：昨日の夕食、来年の流行語、この演習の作者のフルネームなど）を入力して、どのような反応をするか観察してみましょう。

In [ ]:
# --- ここを編集 --- #
question = "2040年のワールドカップはどこで開催されますか？"
# --- 編集ここまで --- #

# 編集禁止セル
if model and tokenizer:
    prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n\n### 応答:\n"
    
    generated_text = generate_text(model, tokenizer, prompt)
    
    print("--- 質問 ---")
    print(question)
    print("\n--- LLMの回答 ---")
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

## まとめ

このノートブックでは、素のLLMの基本的な性質を学びました。

- **回答の揺れ**: 同じ質問でも、実行するたびに異なる回答が生成されることがある。
- **ハルシネーション**: 事実に基づかない、もっともらしい嘘の情報を生成することがある。

これらの性質は、LLMをシステムに組み込む際に考慮すべき重要な課題です。
次の `02_prompting.ipynb` では、プロンプトを工夫することで、これらの振る舞いをある程度制御する方法を学びます。

### メモリ解放
次のノートブックに進む前に、GPUメモリを解放します。

In [ ]:
# 編集禁止セル
import gc
if 'model' in globals() and model is not None: del model
if 'tokenizer' in globals() and tokenizer is not None: del tokenizer
model = None
tokenizer = None
gc.collect()
torch.cuda.empty_cache()
print("モデルを解放し、GPUキャッシュをクリアしました。")